In [1]:
# Load cleaned_airline_data.csv
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
sc= SparkContext()
sqlContext = SQLContext(sc)
df = sqlContext.read.format('com.databricks.spark.csv').options(header='false', inferschema='true').load('cleaned_airline_data.csv')
df.take(1)

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/anderl23/hadoop-3.3.4/spark-3.3.1/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/anderl23/apache-hive-3.1.3-bin/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]


2022-12-15T02:16:42,855 WARN [main] org.apache.spark.util.Utils - Your hostname, DESKTOP-SHBG2MR resolves to a loopback address: 127.0.1.1; using 192.168.0.115 instead (on interface wifi0)
2022-12-15T02:16:42,915 WARN [main] org.apache.spark.util.Utils - Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2022-12-15T02:16:44,233 WARN [Thread-4] org.apache.hadoop.util.NativeCodeLoader - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


/home/anderl23/.local/lib/python3.10/site-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


2022-12-15T02:19:05,449 WARN [Thread-4] org.apache.spark.sql.catalyst.util.package - Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


[Row(_c0=datetime.datetime(2013, 1, 1, 0, 0), _c1='VX', _c2=108, _c3='LAX', _c4='IAD', _c5=700.0, _c6=700.0, _c7=0, _c8=8, _c9=708.0, _c10=1411.0, _c11=7, _c12=1445.0, _c13=1418.0, _c14=-27, _c15=0, _c16=None, _c17=0, _c18=285, _c19=258, _c20=243, _c21=2288, _c22=None, _c23=None, _c24=None, _c25=None, _c26=None)]

In [2]:
#Rename columns
Data_list = ["FL_DATE","OP_CARRIER","OP_CARRIER_FL_NUM","ORIGIN","DEST","CRS_DEPT_TIME","DEPT_TIME","DEP_DELAY",
             "TAXI_OUT","WHEELS_OFF","WHEELS_ON","TAXI_IN","CRS_ARR_TIME","ARR_TIME","ARR_DELAY","CANCELLED",
             "CANCELLEATION_CODE","DIVERTED","CRS_ELAPSED_TIME","ACTUAL_ELAPSED_TIME","AIR_TIME","DISTANCE",
             "CARRIER_DELAY","WEATHER_DELAY","NAS_DELAY","SECURITY_DELAY","LATE_AIRCRAFT_DELAY"]
df = df.toDF(*Data_list)
df.printSchema()

root
 |-- FL_DATE: timestamp (nullable = true)
 |-- OP_CARRIER: string (nullable = true)
 |-- OP_CARRIER_FL_NUM: integer (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- CRS_DEPT_TIME: double (nullable = true)
 |-- DEPT_TIME: double (nullable = true)
 |-- DEP_DELAY: integer (nullable = true)
 |-- TAXI_OUT: integer (nullable = true)
 |-- WHEELS_OFF: double (nullable = true)
 |-- WHEELS_ON: double (nullable = true)
 |-- TAXI_IN: integer (nullable = true)
 |-- CRS_ARR_TIME: double (nullable = true)
 |-- ARR_TIME: double (nullable = true)
 |-- ARR_DELAY: integer (nullable = true)
 |-- CANCELLED: integer (nullable = true)
 |-- CANCELLEATION_CODE: integer (nullable = true)
 |-- DIVERTED: integer (nullable = true)
 |-- CRS_ELAPSED_TIME: integer (nullable = true)
 |-- ACTUAL_ELAPSED_TIME: integer (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- CARRIER_DELAY: integer (nullable = true)
 |-- W

In [3]:
#extract year, month, day of the month and day of the week from Flight date 
import pandas as pd
from pyspark.sql.functions import to_date
from pyspark.sql.functions import year
from pyspark.sql.functions import month
from pyspark.sql.functions import dayofmonth
from pyspark.sql.functions import dayofweek

df = df.withColumn('FL_YEAR',year(df.FL_DATE))
df = df.withColumn('FL_MONTH',month(df.FL_DATE))
df = df.withColumn('FL_DOM',dayofmonth(df.FL_DATE))
df = df.withColumn('FL_DOW',dayofweek(df.FL_DATE))

pd.DataFrame(df.take(5), columns=df.columns).transpose()

,0,1,2,3,4
FL_DATE,2013-01-01 00:00:00,2013-01-01 00:00:00,2013-01-01 00:00:00,2013-01-01 00:00:00,2013-01-01 00:00:00
OP_CARRIER,VX,VX,VX,VX,VX
OP_CARRIER_FL_NUM,108,114,11,121,124
ORIGIN,LAX,LAX,JFK,PHL,LAX
DEST,IAD,IAD,SFO,LAX,PHL
CRS_DEPT_TIME,700.0,2205.0,730.0,700.0,1100.0
DEPT_TIME,700.0,2204.0,729.0,700.0,1104.0
DEP_DELAY,0,-1,-1,0,4
TAXI_OUT,8,12,18,14,12
WHEELS_OFF,708.0,2216.0,747.0,714.0,1116.0


In [4]:
#filter out data from years 2009 and 2010
df1 = df.where("FL_YEAR!=2009")
df2 = df1.where("FL_YEAR!=2010")
df3 = df2.where("FL_YEAR!=2011")
df4 = df3.where("FL_YEAR!=2012")

In [6]:
#descriptive stats
df4.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
OP_CARRIER,43936134,None,None,9E,YX
OP_CARRIER_FL_NUM,43936134,2334.7250371869313,1808.4566645375,1,9855
ORIGIN,43936134,None,None,ABE,YUM
DEST,43936134,None,None,ABE,YUM
CRS_DEPT_TIME,43806048,1329.8598360892997,486.343166781857,1.0,2400.0
DEPT_TIME,43249367,1111.3324493049806,670.8320577201583,-82.0,2710.0
DEP_DELAY,43240548,11.032799491810326,36.8170992001997,-251,2755
TAXI_OUT,43228863,242.56042854978628,544.1838583716815,0,2400
WHEELS_OFF,43225198,1374.276217936584,505.45813345648986,1.0,2400.0


In [7]:
#drop columns with high instances of null values
cols = ("CANCELLEATION_CODE","CARRIER_DELAY","WEATHER_DELAY", "NAS_DELAY", "SECURITY_DELAY", "LATE_AIRCRAFT_DELAY")

df5 = df4.drop(*cols)

In [8]:
#remove rows with null values
df6 = df5.na.drop()

In [9]:
#Uncomment to calculate correlation
import six
for i in df6.columns:
    if( isinstance(df6.select(i).take(1)[0][0], six.integer_types)):
        print( "Correlation to ARR_DELAY for ", i, df6.stat.corr('ARR_DELAY',i))

Correlation to ARR_DELAY for  OP_CARRIER_FL_NUM 0.025600503934606174


Correlation to ARR_DELAY for  DEP_DELAY 0.9511237461302733


Correlation to ARR_DELAY for  TAXI_OUT 0.22201668289481985


Correlation to ARR_DELAY for  TAXI_IN 0.10679503990848845


Correlation to ARR_DELAY for  ARR_DELAY 1.0


...

Correlation to ARR_DELAY for  DISTANCE -0.023173602181855928


Correlation to ARR_DELAY for  FL_YEAR -0.016119419110454766


Correlation to ARR_DELAY for  FL_MONTH -0.011247445798769249


Correlation to ARR_DELAY for  FL_DOM -0.005310493841956384


Correlation to ARR_DELAY for  FL_DOW -0.00273555950547902


In [10]:
df6.write.option("header",True) \
 .csv("cleaned_airline_data_v5.csv")